# Payload/Tool parameter identification

In robotics, the precise identification of tool parameters, such as mass and center of mass (CoM), is crucial for ensuring accurate and safe operation, particularly in tasks that involve dynamic interactions with the environment. Traditional methods of measuring these parameters often require direct physical measurements, which can be both time-consuming and impractical in automated systems. The application of a the developed model offers a significant advantage by enabling the indirect estimation of these parameters through the analysis of predicted forces and torques. this increases the adaptability of the robot to various tools and payloads without the need for frequent recalibration or manual measurements. By accurately estimating the mass and CoM, the system can optimize its performance in tasks such as precision assembly, material handling, and force-controlled operations, ultimately leading to higher productivity and reduced downtime. 

A particularly valuable use case for this technology is in detecting situations where a payload, after being picked up, might slip down or shift position on the tool. Such a shift would alter the mass distribution and center of mass, which can be quickly detected by the virtual sensor. Without this detection capability, the robot could experience unexpected torques and forces, leading to a sudden jerk or unplanned movement, which could compromise the precision of the task or even damage the robot and the payload. By identifying these changes in real-time, the system can adjust its operations to compensate, ensuring smooth and safe handling of the payload.

In this section, a parameter identification is being implemented and the results compared to the real value.

The tool used had a mass of 5.6 kg 

## Estimating Mass 

The estimation of mass using a virtual joint torque sensor is based on the analysis of the forces acting on the tool as it interacts with the environment. The forces predicted by the sensor, typically in the $X$, $Y$, and $Z$ directions, can be used to compute the overall magnitude of the force vector, which, when divided by the gravitational constant, provides an estimate of the tool's mass.

In a static or quasi-static condition, the forces measured by the virtual joint torque sensor are primarily due to the weight of the tool and any attached payload. The relationship between the force components and the mass of the tool can be expressed as:

$$|\vec{F}| = \sqrt{F_x^2 + F_y^2 + F_z^2}$$

where:

- $F_x$, $F_y$, and $F_z$ are the force components in the $X$, $Y$, and $Z$ directions, respectively.
- $|\vec{F}|$ is the magnitude of the resultant force vector.

Given that the force due to gravity is $F = m \cdot g$, where $g$ is the acceleration due to gravity (approximately $9.81 \, \text{m/s}^2$), the mass $m$ can be estimated as:

$$m = \frac{|\vec{F}|}{g}$$

To ensure accuracy, measurements are made over a period of time, and the average of the calculated mass values is taken to provide a more reliable estimate. This approach helps to reduce the impact of sensor noise, and compensate for any variability in the conditions during the measurements, leading to a more accurate estimation of the tool's mass.

This equation assumes that the forces measured are primarily due to gravity, and thus the resultant force vector provides a direct indication of the tool's mass.


In [1]:
import numpy as np


Fx, Fy, Fz = 15.0, -10.0, 35.0  # Example forces in the X, Y, Z directions

# Function to estimate mass from the magnitude of the forces
def estimate_mass(Fx, Fy, Fz):
    # Magnitude of the force vector
    force_magnitude = np.sqrt(Fx**2 + Fy**2 + Fz**2)
    # Estimate mass (assuming F = m * g, where g = 9.81 m/s^2)
    mass = force_magnitude / 9.81
    return mass

# Estimate the mass of the tool
mass = estimate_mass(Fx, Fy, Fz)
print(f"Estimated Mass: {mass:.2f} kg")

Estimated Mass: 4.01 kg


## Estimating Center of Mass
Once the mass $m$ of the tool is known, the Center of Gravity (CoG) coordinates can be calculated using the torques and forces as follows:

The torque $\vec{T}$ generated by a force $\vec{F}$ acting at a distance $\vec{r}$ (which represents the CoG) is given by:

$\vec{T} = \vec{r} \times \vec{F}$

 For each axis, the torque components can be expressed as:

$T_x = y_{com} \cdot F_z - z_{com} \cdot F_y$, 

$T_y = z_{com} \cdot F_x - x_{com} \cdot F_z$, 

$T_z = x_{com} \cdot F_y - y_{com} \cdot F_x$.

Rearranging these equations to solve for the CoG coordinates ($x$, $y$, $z$):

$$
x_{com} = \frac{T_y \cdot F_z - T_z \cdot F_y}{\|F\|^2}
$$

$$
y_{com} = \frac{T_z \cdot F_x - T_x \cdot F_z}{\|F\|^2}
$$

$$
z_{com} = \frac{T_x \cdot F_y - T_y \cdot F_x}{\|F\|^2}
$$

These equations provide the position of the Center of Gravity relative to the origin.


In [3]:
import numpy as np

def calculate_com(Fx, Fy, Fz, Tx, Ty, Tz):
    # Magnitude of the force vector squared
    force_magnitude_squared = Fx**2 + Fy**2 + Fz**2
    
    # Calculate the center of mass coordinates
    x = (Ty * Fz - Tz * Fy) / force_magnitude_squared
    y = (Tz * Fx - Tx * Fz) / force_magnitude_squared
    z = (Tx * Fy - Ty * Fx) / force_magnitude_squared
    
    return x, y, z

# Example usage
Fx, Fy, Fz = 12.5, -9.2, 18.3  # Example forces in Newtons
Tx, Ty, Tz = 1.2, 0.8, -1.5    # Example torques in Newton-meters

x_com, y_com, z_com = calculate_com(Fx, Fy, Fz, Tx, Ty, Tz)

print(f"Center of Mass coordinates: x = {x_com:.4f} m, y = {y_com:.4f} m, z = {z_com:.4f} m")


Center of Mass coordinates: x = 0.0015 m, y = -0.0707 m, z = -0.0365 m
